In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'LTLCDAMSEGTSA/ LAN K RSQ(ONBEG)EA',
 'calculationPrice': 'close',
 'open': 147.7,
 'openTime': 1676395671383,
 'openSource': 'cfaliifo',
 'close': 151.03,
 'closeTime': 1680015329075,
 'closeSource': 'fiaoifcl',
 'high': 152.91,
 'highTime': 1630643774101,
 'highSource': 'acieet ylirdm51 eed nup',
 'low': 147.7,
 'lowTime': 1645706724023,
 'lowSource': 'rcad epueidtnmy5 ie1 le',
 'latestPrice': 155.91,
 'latestSource': 'Close',
 'latestTime': 'July 23, 2021',
 'latestUpdate': 1650062017679,
 'latestVolume': 71694091,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 148.79,
 'delayedPriceTime': 1704727315278,
 'oddLotDelayedPrice': 150.36,
 'oddLotDelayedPriceTime': 1635555106727,
 'extendedPrice': 152.44,
 'extendedChange': -0.29,
 'extendedChangePercent': -0.00196,
 'extendedPriceTime': 1634167428034,
 'previousClose': 147.2,
 'previousVolume': 79346788,
 'change': 1.8,
 

In [5]:
data['latestPrice']
data['marketCap']

2493506191715

In [6]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [7]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,155.91,2493506191715,N/A


In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [9]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,154.23,47026149548,N/A
1,AAL,21.70,14369148544,N/A
2,AAP,218.54,14583192441,N/A
3,AAPL,155.74,2566359527417,N/A
4,ABBV,121.80,212570154687,N/A
...,...,...,...,...
500,YUM,124.64,36702620663,N/A
501,ZBH,167.80,33798570410,N/A
502,ZBRA,561.66,30049802156,N/A
503,ZION,50.89,8687549180,N/A


In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,152.94,46459038294,N/A
1,AAL,22.10,13818467649,N/A
2,AAP,222.94,14674752534,N/A
3,AAPL,151.93,2591415866340,N/A
4,ABBV,118.32,210666941428,N/A
...,...,...,...,...
500,YUM,128.43,37362436634,N/A
501,ZBH,166.17,33830919735,N/A
502,ZBRA,572.52,30804481234,N/A
503,ZION,52.54,8380964478,N/A


Calculating the number of shares to buy

In [24]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:2000000


In [25]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,152.94,46459038294,25
1,AAL,22.10,13818467649,179
2,AAP,222.94,14674752534,17
3,AAPL,151.93,2591415866340,26
4,ABBV,118.32,210666941428,33
...,...,...,...,...
500,YUM,128.43,37362436634,30
501,ZBH,166.17,33830919735,23
502,ZBRA,572.52,30804481234,6
503,ZION,52.54,8380964478,75


Formatting our excel output

In [21]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [16]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)


In [18]:
writer.save()

/Users/shivambhardwaj/opt/miniconda3/envs/keras_env/lib/python3.7/site-packages/xlsxwriter/workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
